# Naive representation pipeline (qiskit)

Main steps:

* input: image of the sky (frequency domain)
* apply 2D Fourier transform --> visibilities (Fourier domain)
* encode classical visibilities data into qbits (non-conventional domain)
* measure qbits (back to conventional domain)
* apply 2D Inverse Fourier transform --> original image?

In [13]:
import numpy as np
import sys
import struct

import qiskit
from qiskit import *
from qiskit.visualization import plot_histogram

## Generating an artificial image of the sky

In [14]:
#image of the sky filled with single precision complex floats
#pixels are set to low complex random values (image background) 
#few pixels are set to larger complex random values in a specified ellipse area (image source/subject)

n = 4
image = np.zeros((n, n), dtype=complex)
image.real = np.random.rand(n , n) / 100
image.imag = np.random.rand(n , n) / 100

h, w = image.shape
mask = circular_mask(h, w, radius=h/2)
sky_image = image.copy()
sky_image[~mask] = complex(np.random.rand() * 100, np.random.rand() * 100)
print(sky_image)

[[8.26709309e+01+9.19763280e+01j 8.26709309e+01+9.19763280e+01j
  7.46427237e-04+6.53659050e-04j 8.26709309e+01+9.19763280e+01j]
 [8.26709309e+01+9.19763280e+01j 4.74646735e-03+2.21801908e-03j
  4.38225442e-03+1.40960349e-03j 3.83196031e-03+5.21136416e-03j]
 [5.93187216e-03+4.65450823e-03j 3.41186895e-03+9.12407671e-03j
  6.21762685e-03+3.49640151e-03j 6.11048626e-03+8.50189677e-04j]
 [8.26709309e+01+9.19763280e+01j 6.11428477e-03+8.86249623e-04j
  6.58547689e-03+1.72597654e-03j 9.67159414e-03+4.16688724e-03j]]


## Applying 2D Fourier transform (visibilities)

In [15]:
visibilities = np.fft.fft2(sky_image)
print(visibilities)

[[ 413.41240507+459.91603682j  248.00279284+275.93169422j
    82.66090795 +91.96581134j  247.99879303+275.92101138j]
 [ 247.99392729+275.92092309j   82.65804095 +91.96932708j
   -82.67550429 -91.97885519j   82.68353235 +91.9752989j ]
 [  82.65801719 +91.97948871j  -82.64644753 -91.97263213j
  -247.9960227 -275.92080669j  -82.67554327 -91.97274376j]
 [ 247.98980755+275.90210174j   82.66635183 +91.97430811j
   -82.67011905 -91.96884673j   82.67395597 +91.97913076j]]


### Sanity check

In [16]:
img = np.fft.ifft2(visibilities)
print(img)
test_real = ((sky_image.real - img.real)**2).mean()
test_imag = ((sky_image.imag - img.imag)**2).mean()

print()
 
print('Reals MSE: '+ str(test_real))
print('Imaginaries MSE: ' + str(test_imag))

[[8.26709309e+01+9.19763280e+01j 8.26709309e+01+9.19763280e+01j
  7.46427237e-04+6.53659050e-04j 8.26709309e+01+9.19763280e+01j]
 [8.26709309e+01+9.19763280e+01j 4.74646735e-03+2.21801908e-03j
  4.38225442e-03+1.40960349e-03j 3.83196031e-03+5.21136416e-03j]
 [5.93187216e-03+4.65450823e-03j 3.41186895e-03+9.12407671e-03j
  6.21762685e-03+3.49640151e-03j 6.11048626e-03+8.50189677e-04j]
 [8.26709309e+01+9.19763280e+01j 6.11428477e-03+8.86249623e-04j
  6.58547689e-03+1.72597654e-03j 9.67159414e-03+4.16688724e-03j]]

Reals MSE: 8.596881362735092e-30
Imaginaries MSE: 1.543105620712343e-29


## Classical data encoding/decoding

In [17]:
number_of_entries = visibilities.shape[0] * visibilities.shape[1]
number_of_bits_real = 32 #since single precision complex floats
number_of_bits_imag = 32
number_of_bits = number_of_bits_real + number_of_bits_imag
off_set = 0

qc = QuantumCircuit(number_of_entries*number_of_bits) #n bits encoded into n qbits (circuit family #1)
for i in range(0, visibilities.shape[0]):
    for j in range(0, visibilities.shape[1]):
            binary_real = float_to_bin_real(visibilities[i, j].real)
            binary_imag = float_to_bin_imag(visibilities[i, j].imag)
            binary = binary_real+binary_imag
            off_set = encoding2(qc, binary, off_set)

#measurement
qc.measure_all()
backend = Aer.get_backend('aer_simulator')
job = backend.run(qc, shots=1, memory=True)
output = job.result().get_memory()[0]
out = reverse(output)

#readout
chunks_real = []
chunks_imag = []
for i in range(0, number_of_entries):
    chunks_real.append(out[number_of_bits*i:(number_of_bits*i)+number_of_bits_real]) #real parts represented every 64 bits starting from the first one
    chunks_imag.append(out[(number_of_bits*i)+number_of_bits_imag:(number_of_bits*i)+number_of_bits_imag+number_of_bits_imag]) #imaginary parts represented every 64 bits starting after the first occurence of a real part  
readout = []
for i in range(0, len(chunks_real)):
    readout.append(complex(bin_to_float_real(chunks_real[i]), bin_to_float_imag(chunks_imag[i]))) 

#reshaping the readout vector into a nxn matrix
readout = np.array(readout).reshape(n , n)
print(readout)

[[ 413.41241455+459.91604614j  248.00279236+275.93170166j
    82.66091156 +91.96581268j  247.99879456+275.92102051j]
 [ 247.993927  +275.92092896j   82.65804291 +91.96932983j
   -82.67550659 -91.97885895j   82.68353271 +91.97529602j]
 [  82.65802002 +91.97949219j  -82.64644623 -91.97263336j
  -247.99601746-275.92080688j  -82.67554474 -91.97274017j]
 [ 247.98980713+275.90209961j   82.66635132 +91.97431183j
   -82.67012024 -91.96884918j   82.67395782 +91.97913361j]]


## Applying 2D Inverse Fourier transform (+ fidelity test)

In [18]:
img = np.fft.ifft2(readout)
print(img)
test_real = ((sky_image.real - img.real)**2).mean()
test_imag = ((sky_image.imag - img.imag)**2).mean()

print()

print('Reals MSE: '+ str(test_real))
print('Imaginaries MSE: ' + str(test_imag))

[[8.26709323e+01+9.19763303e+01j 8.26709313e+01+9.19763293e+01j
  7.47203827e-04+6.52790070e-04j 8.26709313e+01+9.19763293e+01j]
 [8.26709313e+01+9.19763293e+01j 4.74596024e-03+2.21776962e-03j
  4.38165665e-03+1.40905380e-03j 3.83234024e-03+5.21230698e-03j]
 [5.93328476e-03+4.65631485e-03j 3.41272354e-03+9.12427902e-03j
  6.21938705e-03+3.49664688e-03j 6.10971451e-03+8.50200653e-04j]
 [8.26709313e+01+9.19763293e+01j 6.11543655e-03+8.86440277e-04j
  6.58655167e-03+1.72567368e-03j 9.67264175e-03+4.16707993e-03j]]

Reals MSE: 8.608877671981738e-13
Imaginaries MSE: 1.1331686939524147e-12


## Utils

### Quantum

In [19]:
def encoding2(qc, binary, off_set):
    
    for i in range(0, len(binary)):
        qc.reset(off_set+i)

        if binary[i]=='1':
            qc.x(off_set+i)
    
    off_set += len(binary)

    return off_set

### Classical

In [20]:
#float to binary / binary to float
def float_to_bin_real(num):
    return format(struct.unpack('!I', struct.pack('!f', num))[0], '032b')
def float_to_bin_imag(num):
    return format(struct.unpack('!I', struct.pack('!f', num))[0], '032b')

def bin_to_float_real(binary):
    return struct.unpack('!f',struct.pack('!I', int(binary, 2)))[0]
def bin_to_float_imag(binary):
    return struct.unpack('!f',struct.pack('!I', int(binary, 2)))[0]

#reverse a string (for the measurement step readout)
def reverse(string):
    string = string[::-1]
    return string

#creates a circular mask over a 2D array
def circular_mask(h, w, center=None, radius=None):
    if center is None: #image center
        center = (int(w/2), int(h/2))
    if radius is None: #smallest distance between center and image bounderies
        radius = min(center[0], center[1], w-center[0], h-center[1])
        
    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)
    mask = dist_from_center <= radius
    
    return mask